In [1]:
import pandas as pd
import urllib.request as req
import requests
import ast

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

census_key = 'a4feafd1e894848eda9a783376cf7977222aecd2' ## The key from https://www.census.gov/data/developers.html "Request a key".

# APIs from the US Census Bureau
population_density_url = 'https://api.census.gov/data/2019/pep/population?get=DENSITY&in=state:*&for=county:*&key={}'.format(census_key)
above_65_url = 'https://api.census.gov/data/2018/acs/acs5/subject?get=GEO_ID,NAME,S0101_C02_030E&in=state:*&for=county:*&key={}'.format(census_key)
transport_url = 'https://api.census.gov/data/2018/acs/acs5?get=GEO_ID,NAME,B08006_001E,B08006_003E,B08006_017E&in=state:*&for=county:*&key={}'.format(census_key)
race_url = 'https://api.census.gov/data/2018/acs/acs5/profile?get=GEO_ID,NAME,DP05_0070E,DP05_0077E&in=state:*&for=county:*&key={}'.format(census_key)
poverty_url = 'https://api.census.gov/data/2018/acs/acs5/subject?get=GEO_ID,NAME,S1701_C03_001E&in=state:*&for=county:*&key={}'.format(census_key)
income_url = 'https://api.census.gov/data/2018/acs/acs5/subject?get=GEO_ID,NAME,S1902_C03_019E&in=state:*&for=county:*&key={}'.format(census_key)
employed_url = 'https://api.census.gov/data/2018/acs/acs5/subject?get=GEO_ID,NAME,S2301_C03_001E&in=state:*&for=county:*&key={}'.format(census_key)
unemployment_url = 'https://api.census.gov/data/2018/acs/acs5/subject?get=GEO_ID,NAME,S2301_C04_001E&in=state:*&for=county:*&key={}'.format(census_key)
mean_commute_url = 'https://api.census.gov/data/2018/acs/acs5/subject?get=GEO_ID,NAME,S0801_C01_046E&in=state:*&for=county:*&key={}'.format(census_key)
labour_url = 'https://api.census.gov/data/2018/acs/acs5/profile?get=GEO_ID,NAME,DP03_0028PE,DP03_0030PE,DP03_0031PE&in=state:*&for=county:*&key={}'.format(census_key)
insurance_url = 'https://api.census.gov/data/2018/acs/acs5/profile?get=GEO_ID,NAME,DP03_0099PE&in=state:*&for=county:*&key={}'.format(census_key)

# links to the JHU GitHub
jhu_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
jhu_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

In [2]:
def get_census_data(url):
    ''' A function to get the data from USA Census Bureau
        arguments:
            url: the url for the API call
        return:
            df: the dataframe with the pulled data
    '''
    df = pd.DataFrame(ast.literal_eval(req.urlopen(req.Request(url)).read().decode('utf8').replace('null', '"1"')))
    df.columns = df.iloc[0]
    df = df[1:]
    df['FIPS'] = df.pop('GEO_ID').apply(lambda x: int(x[-5:]))
    return df

def exportjson(df, name):
    ''' A function to export to json files
        arguments:
            df: the dataframe to export
            name: the name of the file without the '.json'
    '''
    df_json = df[['code', 'name', name]]
    df_json.rename(columns={name: 'value'}, inplace=True)
    with open('../Maps/data/US/'+name+'.json', 'w') as file:
        file.write(df_json.to_json(orient='records'))
    print('10 percentile ('+name+'): {}'.format(df[df['Density'] > 0][name].quantile(0.10)))
    print('90 percentile ('+name+'): {}'.format(df[df['Density'] > 0][name].quantile(0.90)))
    print('')

In [3]:
## The density data from census.gov
density = pd.DataFrame(ast.literal_eval(req.urlopen(req.Request(population_density_url)).read().decode('utf8').replace('null', '"0"')))
density.columns = density.iloc[0]
density = density[1:]
density.DENSITY = density.DENSITY.astype('float32')
density.state = density.state.astype('string')
density.county = density.county.astype('string')
density['FIPS'] = density['state'] + density['county']
density.FIPS = density.FIPS.astype('int32')
density = density.drop(labels=['state', 'county'], axis=1)
columns = density.columns.tolist()
columns = columns[::-1]
density = density[columns]
density.rename(columns={'DENSITY': 'Density'}, inplace=True)

In [4]:
# The FIPS and state names
df_census = get_census_data(above_65_url)
df_census = df_census[['FIPS', 'NAME']]
df_census['County'] = df_census['NAME'].str.split(', ').str[0]
df_census['State'] = df_census.pop('NAME').str.split(', ').str[1]
df_census = df_census.sort_values(df_census.columns[0], ignore_index=True)
df_census = pd.merge(df_census, density, on='FIPS')

# 2018 5 year ACS data on Age
dft = get_census_data(above_65_url)
dft = dft[['FIPS', 'S0101_C02_030E']]
dft.rename(columns={'S0101_C02_030E': 'Senior Citizen'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Senior Citizen'] = df_census['Senior Citizen'].apply(lambda x: float(x))/100.

# 2018 5 year ACS data on transport used
dft = get_census_data(transport_url)
dft['Transit-1'] = 1. - (dft['B08006_003E'].astype('float')+dft['B08006_017E'].astype('float'))/dft['B08006_001E'].astype('float')
dft = dft[['FIPS', 'Transit-1']]
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Transit'] = df_census.pop('Transit-1')
df_census.at[df_census[df_census['FIPS']==35039].index, 'Transit'] = 0.159 ## Imputed value from 2011 since values are null in 2018

# 2018 5 year ACS data on non-white
dft = get_census_data(race_url)
dft['Non-White'] = 1. - dft['DP05_0077E'].astype('float')/dft['DP05_0070E'].astype('float')
dft = dft[['FIPS', 'Non-White']]
df_census = pd.merge(df_census, dft, on='FIPS')

# 2018 5 year ACS data on Poverty
dft = get_census_data(poverty_url)
dft = dft[['FIPS', 'S1701_C03_001E']]
dft.rename(columns={'S1701_C03_001E': 'Poverty-1'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Poverty'] = df_census.pop('Poverty-1').apply(lambda x: float(x))/100.
df_census.at[df_census[df_census['FIPS']==35039].index, 'Poverty'] = 0.237 ## Imputed value from 2011 since values are null in 2018

# 2018 5 year ACS data on Income per capita
dft = get_census_data(income_url)
dft = dft[['FIPS', 'S1902_C03_019E']]
dft.rename(columns={'S1902_C03_019E': 'Income/Capita'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Income/Capita'] = df_census['Income/Capita'].apply(lambda x: float(x))
df_census.at[df_census[df_census['FIPS']==35039].index, 'Income/Capita'] = 19678 ## Imputed value from 2011 since values are null in 2018

# 2018 5 year ACS data on Employment
dft = get_census_data(employed_url)
dft = dft[['FIPS', 'S2301_C03_001E']]
dft.rename(columns={'S2301_C03_001E': 'Employed-1'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Employed'] = df_census.pop('Employed-1').apply(lambda x: float(x))/100.
df_census.at[df_census[df_census['FIPS']==35039].index, 'Employed'] = 0.382287 ## Imputed value from 2011 since values are null in 2018

# 2018 5 year ACS data on Unemployment
dft = get_census_data(unemployment_url)
dft = dft[['FIPS', 'S2301_C04_001E']]
dft.rename(columns={'S2301_C04_001E': 'Unemployment-1'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Unemployment'] = df_census.pop('Unemployment-1').apply(lambda x: float(x))/100.
df_census.at[df_census[df_census['FIPS']==35039].index, 'Unemployment'] = 0.114 ## Imputed value from 2011 since values are null in 2018

# 2018 5 year ACS data on Mean Commute in minutes
dft = get_census_data(mean_commute_url)
dft = dft[['FIPS', 'S0801_C01_046E']]
dft.rename(columns={'S0801_C01_046E': 'MeanCommute'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['MeanCommute'] = df_census['MeanCommute'].apply(lambda x: float(x))
df_census.at[df_census[df_census['FIPS']==35039].index, 'MeanCommute'] = 26.2 ## Imputed value from 2011 since values are null in 2018
df_census.at[df_census[df_census['FIPS']==48301].index, 'MeanCommute'] = 25.4 ## Imputed value from 2011 since values are null in 2018

# 2018 5 year ACS data on Labour
dft = get_census_data(labour_url)
dft['Labour'] = (dft['DP03_0028PE'].astype('float') + dft['DP03_0030PE'].astype('float') + dft['DP03_0031PE'].astype('float'))/100.
dft = dft[['FIPS', 'Labour']]
df_census = pd.merge(df_census, dft, on='FIPS')
df_census.at[df_census[df_census['FIPS']==35039].index, 'Labour'] = 0.271 + 0.108 + 0.07 ## Imputed value from 2011 since values are null in 2018

# 2018 5 year ACS data on Health Insurance
dft = get_census_data(insurance_url)
dft = dft[['FIPS', 'DP03_0099PE']]
dft.rename(columns={'DP03_0099PE': 'Uninsured'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Uninsured'] = df_census.pop('Uninsured').apply(lambda x: float(x))/100.

In [5]:
# generate the data files in the correct format
data_map = df_census.copy(deep=True)
data_map['FIPS'] = data_map['FIPS'].apply(str).apply(lambda x: '{0:0>5}'.format(x)).apply(lambda x : x[2:])
data_map['State'] = data_map['State'].apply(lambda x: us_state_abbrev[x].lower())
data_map['code'] = 'us-'+data_map['State']+'-'+data_map['FIPS']
data_map['name'] = data_map['County']+', '+data_map['State'].apply(lambda x: x.upper())

# Non-White
data_map['Non-White'] = data_map['Non-White']*100.
exportjson(data_map, 'Non-White')

# Income
data_map['Income'] = data_map['Income/Capita']
exportjson(data_map, 'Income')

# Poverty
data_map['Poverty'] = data_map['Poverty']*100.
exportjson(data_map, 'Poverty')

# Unemployment
data_map['Unemployment'] = data_map['Unemployment']*100.
exportjson(data_map, 'Unemployment')

# Uninsured
data_map['Uninsured'] = data_map['Uninsured']*100.
exportjson(data_map, 'Uninsured')

# Employed
data_map['Employed'] = data_map['Employed']*100.
exportjson(data_map, 'Employed')

# Labour
data_map['Labour'] = data_map['Labour']*100.
exportjson(data_map, 'Labour')

# Transit
data_map['Transit'] = data_map['Transit']*100.
exportjson(data_map, 'Transit')

# Mean Commute
exportjson(data_map, 'MeanCommute')

# Senior Citizen
data_map['Senior Citizen'] = data_map['Senior Citizen']*100.
exportjson(data_map, 'Senior Citizen')

exportjson(data_map, 'Density')

10 percentile (Non-White): 4.570574181924518
90 percentile (Non-White): 54.597917169690504

10 percentile (Income): 20168.800000000003
90 percentile (Income): 34470.700000000004

10 percentile (Poverty): 8.4
90 percentile (Poverty): 24.0

10 percentile (Unemployment): 2.7
90 percentile (Unemployment): 9.0

10 percentile (Uninsured): 4.7
90 percentile (Uninsured): 16.8

10 percentile (Employed): 44.1
90 percentile (Employed): 65.1

10 percentile (Labour): 38.099999999999994
90 percentile (Labour): 56.190000000000005

10 percentile (Transit): 10.275783364747099
90 percentile (Transit): 20.761741402036087

10 percentile (MeanCommute): 16.5
90 percentile (MeanCommute): 31.0

10 percentile (Senior Citizen): 12.9
90 percentile (Senior Citizen): 24.3

10 percentile (Density): 4.0626819133758545
90 percentile (Density): 408.30596008300785



In [50]:
# The data file for the death rates
deaths = pd.read_csv(jhu_deaths)
deaths.drop(deaths[(deaths['FIPS'] < 1000) | (deaths['FIPS'] > 57000)].index , inplace=True)
deaths.dropna(inplace=True)

data_deaths = deaths.copy(deep=True)
data_deaths['FIPS'] = data_deaths['FIPS'].apply(int).apply(str).apply(lambda x: '{0:0>5}'.format(x)).apply(lambda x : x[2:])
data_deaths['Province_State'] = data_deaths['Province_State'].apply(lambda x: us_state_abbrev[x].lower())
data_deaths['value'] = round((data_deaths['8/16/20']/data_deaths['Population']*100000).apply(int))
data_deaths['code'] = 'us-'+data_deaths['Province_State']+'-'+data_deaths['FIPS']
data_deaths['name'] = data_deaths['Admin2']+' County, '+data_deaths['Province_State'].apply(lambda x: x.upper())
data_deaths = data_deaths[['code', 'name', 'value']]
with open('../Maps/data/US/deaths.json', 'w') as file:
    file.write(data_deaths.to_json(orient='records'))
    
print('10 percentile (death rate): {}'.format(data_deaths['value'].quantile(0.10)))
print('90 percentile (death rate): {}'.format(data_deaths['value'].quantile(0.90)))

10 percentile (death rate): 0.0
90 percentile (death rate): 75.90000000000009


In [51]:
# The data file for the confirmed case rate
confirmed = pd.read_csv(jhu_confirmed)
confirmed.drop(confirmed[(confirmed['FIPS'] < 1000) | (confirmed['FIPS'] > 57000)].index , inplace=True)
confirmed.dropna(inplace=True)
confirmed['Population'] = deaths['Population']

data_confirmed = confirmed.copy(deep=True)
data_confirmed['FIPS'] = data_confirmed['FIPS'].apply(int).apply(str).apply(lambda x: '{0:0>5}'.format(x)).apply(lambda x : x[2:])
data_confirmed['Province_State'] = data_confirmed['Province_State'].apply(lambda x: us_state_abbrev[x].lower())
data_confirmed['value'] = round((data_confirmed['8/16/20']/data_confirmed['Population']*100000).apply(int))
data_confirmed['code'] = 'us-'+data_confirmed['Province_State']+'-'+data_confirmed['FIPS']
data_confirmed['name'] = data_confirmed['Admin2']+' County, '+data_confirmed['Province_State'].apply(lambda x: x.upper())
data_confirmed = data_confirmed[['code', 'name', 'value']]
with open('../Maps/data/US/confirmed.json', 'w') as file:
    file.write(data_confirmed.to_json(orient='records'))
    
print('10 percentile (confirmed rate): {}'.format(data_confirmed['value'].quantile(0.10)))
print('90 percentile (confirmed rate): {}'.format(data_confirmed['value'].quantile(0.90)))

10 percentile (confirmed rate): 228.0
90 percentile (confirmed rate): 2628.800000000001
